In [1]:
# MIGRATE THESE TO A SCRIPT
import sys
import os
import time
import re
dat_dir = '/Workspace/CMB/Data/'
temp = "/Workspace/CMB/Data/temp"

In [2]:
pat_list = [dat_dir+p for p in os.listdir(dat_dir) if re.match(r'^\d+$', p) is not None]
len(pat_list)

450

In [ ]:
# 스트리핑 된 거 한 폴더에 몰아주기
# if not os.path.isdir(temp) : os.makedirs(temp)
# for path in pat_list:
#     base = !basename $path
#     content = !ls $path
#     if "fsl_preproc.nii.gz" in content:
#         to_move = path+"/fsl_preproc.nii.gz"
#         name = temp+"/"+base[0]+"_preproc.nii.gz"
#         !cp $to_move $name

In [34]:
# CORRECTING MY WRONG DOING! :(  ----> use it for renaming
# things = os.listdir(temp)
# for thing in things :
#     before = temp+"/"+thing
#     after = thing.strip(".nii.gz").strip("fsl_")
#     after = temp+"/"+after+"_preproc.nii.gz"
#     !mv $before $after
# comment out after usage!!!!!!

In [3]:
out = "/Workspace/CMB/Data/result"
if not os.path.isdir(out) : os.makedirs(out)
mName = "Microbleednet"
mPath = "/sundara_pretrained/"
mFull = mPath + mName
true = "True"
cp = "best"
cpn = "" # not used
!microbleednet evaluate -i $temp -p $true -o $out -m $mFull -cp_type $cp

/opt/conda/bin/microbleednet:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  __import__('pkg_resources').run_script('microbleednet==1.0.1', 'microbleednet')


classes:2, pretrained:True, name:Microbleednet, cp:best
subject: {'inp_path': '/Workspace/CMB/Data/temp/21102103_preproc.nii.gz', 'basename': '21102103'}
/opt/conda/lib/python3.11/site-packages/microbleednet-1.0.1-py3.11.egg/microbleednet/microbleed_net/microbleednet_data_preparation.py:472: RuntimeWarning: invalid value encountered in divide
/opt/conda/lib/python3.11/site-packages/microbleednet-1.0.1-py3.11.egg/microbleednet/microbleed_net/microbleednet_data_preparation.py:475: RuntimeWarning: invalid value encountered in divide
crp dat:230, crp lbls: 230, crp brain:230, crp frst:230
TEST MODE: WHICH IS MY GOAL
from data prep: datlen:120, lablen:120, pws:120
subject_dat length: 3
Traceback (most recent call last):
  File "/opt/conda/bin/microbleednet", line 4, in <module>
    __import__('pkg_resources').run_script('microbleednet==1.0.1', 'microbleednet')
  File "/opt/conda/lib/python3.11/site-packages/pkg_resources/__init__.py", line 722, in run_script
    self.require(requires)[0].ru

-- BELOW ARE TEST CELLS --

In [83]:
import numpy as np
import torch
import nibabel as nib
from scipy.ndimage import gaussian_filter as gf

In [67]:
a = [np.arange(2).astype(float), np.array([2,3,4]).astype(float)]
b = [torch.tensor([5,6,4])]

In [68]:
print(f'test:{a}')
print(f'test:{b}')

test:[array([0., 1.]), array([2., 3., 4.])]
test:[tensor([5, 6, 4])]


In [71]:
img = nib.load("/Workspace/CMB/Data/temp/21102001_preproc.nii.gz").get_fdata()

In [73]:
label = np.zeros_like(img, dtype=int)

In [77]:
brain = (img > 0).astype(int)

In [85]:
def frsx(input_image, radii2d, alpha=2, factor_std=0.1, bright=False, dark=False):
    [gx, gy] = np.gradient(input_image)
    maximum_radius = np.ceil(np.max(radii2d))
    offset_img = np.array([maximum_radius, maximum_radius]).astype(int)
    rad_sym_output = np.zeros(input_image.shape + 2 * offset_img)

    Sum_sym = np.zeros([len(radii2d), rad_sym_output.shape[0], rad_sym_output.shape[1]])
    rad_index=0
    for n in radii2d:
        O_n = np.zeros(rad_sym_output.shape)
        M_n = np.zeros(rad_sym_output.shape)
        for i in range(0, input_image.shape[0]):
            for j in range(0, input_image.shape[1]):
                p = np.array([i, j]).astype(int)
                g = np.array([gx[i, j], gy[i, j]]).astype(int)
                g_norm = np.sqrt(g @ g.T)
                if (g_norm > 0):
                    gp = np.round((g // g_norm) * n)
                    if bright:
                        ppos = p+ gp
                        ppos = (ppos + offset_img)
                        O_n[int(ppos[0]), int(ppos[1])] = O_n[int(ppos[0]), int(ppos[1])] + 1
                        M_n[int(ppos[0]), int(ppos[1])] = M_n[int(ppos[0]), int(ppos[1])] + g_norm
                    if dark:
                        pneg = p - gp
                        pneg = (pneg + offset_img)
                        O_n[int(pneg[0]), int(pneg[1])] = O_n[int(pneg[0]), int(pneg[1])] - 1
                        M_n[int(pneg[0]), int(pneg[1])] = M_n[int(pneg[0]), int(pneg[1])] - g_norm
        O_n = abs(O_n)
        O_n = O_n / np.max(O_n)
        M_n = abs(M_n)
        M_n = M_n / np.max(M_n)
        S_n = (O_n ** alpha) * M_n
        Sum_sym[rad_index, :, :] = gf(S_n, n * factor_std) ## depric
        rad_index = rad_index + 1
    rad_sym_output = np.squeeze(np.sum(Sum_sym, axis=0))
    rad_sym_output = rad_sym_output[offset_img[0]:-offset_img[1], offset_img[0]:-offset_img[1]]
    return rad_sym_output
def get_frst_data(data):
    radii2d = np.array([2, 3])
    frst_output = np.zeros([data.shape[0], data.shape[1], data.shape[2]])
    for i in range(data.shape[2]):
        inp_slice = data[:, :, i]
        feature_frst = frsx(inp_slice, radii2d, alpha=1, factor_std=0.1, bright=False, dark=True)
        frst_output[:, :, i] = feature_frst
    return frst_output
                        

In [86]:
frst = get_frst_data(img)

/tmp/ipykernel_656/899793789.py:30: RuntimeWarning: invalid value encountered in divide
  O_n = O_n / np.max(O_n)
/tmp/ipykernel_656/899793789.py:32: RuntimeWarning: invalid value encountered in divide
  M_n = M_n / np.max(M_n)


In [89]:
frst[np.isnan(frst)] = 0

In [100]:
c = "abc"
c == "abc"
if c == "dd":
    print("one")
elif c == "ee":
    print("two")
else:
    print("trhee")

trhee
